+ 혁펜하임의 ATT강의를 듣고 참고하여 정리한 내용.

# GPT-1

## Architecture / Training

__G__enerative __P__re-trained __T__ransformer

<img src="https://drive.google.com/uc?id=1NCMYmuu27K40G1GoaYJh_KrrJj6711RC
" width=400> <img src="https://drive.google.com/uc?id=1-ofi9scDOvZ_WnMZUF5uKRyzDDXNrOvq" width=300>

기존의 transformer에서 디코더만 뚝 떼어낸 모델. 자연스럽게 attention layer하나가 사라졌다.

GPT는 오로지 Next Token Prediction만 한다. __이를 통해 언어를 이해한다__

+ 원래의 문장을 입력받아서, 한칸 밀린 문장을 예측한다.
    + ex) 나는 대학생 입니다 -> GPT -> 대학생 입니다 \<eos>
+ 여기에 masked self-attention을 하게되는데, 앞의 단어들만을 통해서 뒷 단어를 생성하도록 학습되는 것이다.
+ __문장만 갖고 있으면 되며, label이 필요 없는 self-supervised 학습이라는 장점!__

BookCorpus라는 데이터셋(약 7400만 문장)을 이용해, pre-training하고, 이후에 downstream task로 fine-tuning한다.

## Fine-tuning

기존의 next token prediction loss를 $L_1$이라하고(좌측),  
이후의 downstream task를 위한 loss를 $L_2$라하면(우측),  
$L = \lambda L_1 + L_2$, $\lambda=0.5$를 loss로 사용함.

애초에 $L_1$(다음토큰예측)을 목적으로 사전 학습된 모델이기 때문에, 해당 loss가 사전에 모델에 대한 prior에 해당하는 term이라 생각하면 regularization처럼 작용하게 된다. 따라서, 오버피팅을 방지할 수 있게된다.

<img src="https://drive.google.com/uc?id=1g84i0FXwXhnqg81mFjXtjjmEb1LavkIL" height=400>

그리고 약간은 multi-task learning같기도..?  
어쨌든, downstream task를 위한 새로운 linear 계층을 추가해준다. 각 task마다 조금씩 다른 구조를 가진다.

다만 중점적으로 볼 것은 Extract token이다.  
모델이 next token prediction을 하기 때문에 Masked self attention을한다.   
이 때, 각 토큰은 이전의 단어 정보를 포함한 토큰인데, 모델 output으로 나오는 토큰들을 보면 결국 맨 마지막 토큰을 제외하고는 문장 전체를 포함해서 attetion하지 않는다.   
따라서, input에 대한 정보를 모두 포함하고 있는 Extract token을 이용해서 linear 레이어에 넣어 원하는 task를 수행하는 것이다.

__BERT나 ViT에서 사용하는 cls 토큰의 역할을 수행.__

## Experiments

<img src="https://drive.google.com/uc?id=1izVmCoIhoYyT6BLPP4mDs87nfODkxnfV" height=450>

위의 것은 질문에 대한 답을 본문이 함축하고 있는지 알아보는 task  
밑의 것은 Question answering. (객관식 문제풀이라 보면됨.)  



각 모델에 5x, 9x이거는 그 수만큼 ensemble했다는 의미.

기존 모델들 대비 다 sota 달성

<img src="https://drive.google.com/uc?id=1MbSJUMxfUBz0QlMFZDeWo6eZ8u2e7sTg" height=400>

파라미터 수가 늘어날수록 성능이 계속해서 좋아짐.  
__LLM의 등장 예고!__

<img src="https://drive.google.com/uc?id=1HcLflxh_ovoCBEswC81_-sBw54n7Hkd8" height=250>

앞의 4가지 task는 수천단위의 작은 데이터셋, 뒤의 4가지는 수십만단위의 큰 데이터셋이다.  
이 때, 작은 데이터셋의 경우에는 $L_2$만 고려한 것이 더 좋았다.(당연히도 적은 데이터셋일 때 regularization할 여력도 없을 것임..)

특별히, pre-train을 안했을 때에는 성능이 현저히 떨어졌었다.   
즉, 방대한 데이터셋을 통해 next token prediction으로 언어에 대한 이해를 해놓는 것이 중요한 의미를 갖는다고 생각할 수 있다.
